In [ ]:
import cv2
import numpy as np
import os

def video_to_frames(video_path, frame_size=(224, 224), max_frames=16):
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, frame_size)  
        frames.append(frame)
        
        if len(frames) >= max_frames:
            break

    cap.release()
    
    # Nếu video ngắn hơn max_frames, lặp lại frame cuối
    while len(frames) < max_frames:
        frames.append(frames[-1])

    return np.array(frames)  


In [5]:
def load_dataset(dataset_path):
    X, y = [], []
    
    for label, category in enumerate(["NonViolence", "Violence"]):
        folder_path = os.path.join(dataset_path, category)
        for filename in os.listdir(folder_path):
            video_path = os.path.join(folder_path, filename)
            frames = video_to_frames(video_path)  # Chuyển video thành tensor 3D
            X.append(frames)
            y.append(label)  # 1 = Violence, 0 = Non-Violence

    return np.array(X), np.array(y)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Load dữ liệu
dataset_path = "./dataset"
X, y = load_dataset(dataset_path)

# Chuyển đổi nhãn thành dạng one-hot encoding
y = np.array(y)

# Chia dữ liệu train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Chuẩn hóa dữ liệu
X_train, X_test = X_train / 255.0, X_test / 255.0

# Xây dựng mô hình CNN 3D
model = Sequential([
    Conv3D(32, kernel_size=(3,3,3), activation="relu", input_shape=(16,224,224,3)),
    MaxPooling3D(pool_size=(1,2,2)),
    Conv3D(64, kernel_size=(3,3,3), activation="relu"),
    MaxPooling3D(pool_size=(1,2,2)),
    Conv3D(128, kernel_size=(3,3,3), activation="relu"),
    MaxPooling3D(pool_size=(2,2,2)),
    Flatten(),
    Dense(256, activation="relu"),
    Dropout(0.5),
    Dense(1, activation="sigmoid")  # 1 = Bạo lực, 0 = Không bạo lực
])

model.compile(optimizer=Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

print(model.summary())  # Kiểm tra kiến trúc mô hình
model.fit(X_train, y_train, epochs=15, batch_size=4, validation_data=(X_test, y_test))

model.save("3k-224-16.keras",save_format="keras")  # Lưu mô hình


In [ ]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)  # Chuyển đổi xác suất thành nhãn (0 hoặc 1)

# Tính toán các chỉ số đánh giá
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print("Confusion Matrix:")
print(conf_matrix)